<div class=title>
<strong><center><font size="3">DAY 11 - IYKRA</font></center>
<hr>
<center><font size="5"><strong>Self-Practice 4</strong></font></center>
<center><font size="4"><strong>Foundation of Machine Learning</strong></font></center>
<hr>
<p style="text-align:center">Author</p>
<center>Joshua Effendi</center>
<p style="text-align:center">Date:</p>
<center>4 November 2019</center></strong>
</div>

<div class=title1>
    <b><font size='3'>Import Libraries</font></b>
</div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from datetime import datetime
from xgboost import XGBClassifier

from IPython.core.display import HTML

def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

<div class=title1>
    <b><font size='3'>Import Data</font></b>
</div>

In [ ]:
df_train = pd.read_csv('training_dataset.csv')
df_train

In [ ]:
df_test = pd.read_csv('test_dataset.csv')
df_test.head()

<div class=title1>
    <b><font size='3'>Data Cleansing</font></b>
</div>

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

<div class=cbox>
    <b>From the train data, there are lots of missing data so we have to clean the data. Those features which has 20% more missing data will be dropped.</b>
</div>

In [ ]:
df_train = df_train.drop(columns=['departure_airport_gate',
                                 'departure_airport_terminal',
                                 'arrival_airport_gate',
                                 'arrival_airport_terminal'])

<div class=cbox>
    <b>From the train data, drop features in which are not related to delay as business sense (logic).</b>
</div>

In [ ]:
df_train_c = df_train.drop(columns=['id',
                                    'number',
                                    'airline_name',
                                    'departure_airport_code',
                                    'departure_airport_country',
                                    'departure_airport_region',
                                    'departure_airport_timezone',
                                   'arrival_airport_code',
                                   'arrival_airport_country',
                                   'arrival_airport_region',
                                   'arrival_airport_timezone',
                                   'flight_equipment_iata',
                                   'flight_equipment_name',
                                   'flight_duration'])
df_train_c.head()

<div class=cbox>
    <b>From the train data, drop rows in which are still Nan.</b>
</div>

In [ ]:
df_train_c = df_train_c.drop(index = df_train_c[df_train_c['scheduled_arrival_time'].isnull()].index).reset_index(drop=True)
df_train_c.isnull().sum()

<div class=cbox>
    <b>Change data type of object into DateTime.</b>
</div>

In [ ]:
df_train_c['scheduled_departure_time'] = pd.to_datetime(df_train_c['scheduled_departure_time'], format='%Y-%m-%d')
df_train_c['scheduled_arrival_time'] = pd.to_datetime(df_train_c['scheduled_arrival_time'], format='%Y-%m-%d')

<div class=cbox>
    <b>Convert DateTime into several parts; Departure Hour and Day. Get the day because it has an effect on delay</b>
</div>

In [ ]:
df_train_c1 = df_train_c.copy()
df_train_c1 = df_train_c1.drop(columns = ['arrival_airport_city','arrival_airport_name','departure_airport_city'])
df_train_c1.head()

In [ ]:
df_train_c1['departure_hour'] = df_train_c1.scheduled_departure_time.apply(lambda x: x.hour)
df_train_c1['departure_day'] = df_train_c1.scheduled_departure_time.apply(lambda x: x.strftime('%A'))
df_train_c1['arival_hour'] = df_train_c1.scheduled_arrival_time.apply(lambda x: x.hour)
df_train_c1 = df_train_c1.drop(columns=['scheduled_departure_time','scheduled_arrival_time'])
df_train_c1

<div class=title1>
    <b><font size='3'>Data Test Cleaning</font></b>
</div>

<div class=cbox>
    <b>Do the data cleaning from data train to data test</b>
</div>

In [ ]:
df_test = df_test.drop(columns=['departure_airport_gate',
                               'departure_airport_terminal',
                               'arrival_airport_gate',
                               'arrival_airport_terminal',
                               'id',
                               'number',
                               'airline_name',
                               'departure_airport_code',
                               'departure_airport_country',
                               'departure_airport_region',
                               'departure_airport_timezone',
                               'arrival_airport_code',
                               'arrival_airport_country',
                               'arrival_airport_region',
                               'arrival_airport_timezone',
                               'flight_equipment_iata',
                               'flight_equipment_name',
                               'flight_duration',
                               'arrival_airport_city',
                               'arrival_airport_name',
                               'departure_airport_city'])

In [ ]:
df_test['scheduled_departure_time'] = pd.to_datetime(df_test['scheduled_departure_time'], format='%Y-%m-%d')
df_test['scheduled_arrival_time'] = pd.to_datetime(df_test['scheduled_arrival_time'], format='%Y-%m-%d')

In [ ]:
df_test = df_train_c.drop(index = df_test[df_test['scheduled_arrival_time'].isnull()].index).reset_index(drop=True)
df_test.isnull().sum()

In [ ]:
df_test['departure_hour'] = df_test.scheduled_departure_time.apply(lambda x: x.hour)
df_test['departure_day'] = df_test.scheduled_departure_time.apply(lambda x: x.strftime('%A'))
df_test['arival_hour'] = df_test.scheduled_arrival_time.apply(lambda x: x.hour)
df_test = df_test.drop(columns=['scheduled_departure_time','scheduled_arrival_time'])
df_test.head()

<div class=title1>
    <b><font size='3'>Data Processing</font></b>
</div>

<div class=title1>
    <b><font size='2'>Provide travel recommendation to passenger (best time to take a flight or best airline), so they will
not get caught in delay. Each recommendation has to be supported by at least 1 graph.</font></b>
</div>

In [ ]:
df_train_c1_ga = df_train_c.groupby('airline').mean().sort_values('delay')[0:10]

In [ ]:
sns.barplot(df_train_c1_ga.index, df_train_c1_ga['delay'])

<div class=cbox>
    <b>From the graph above, there are 5 airlines that have 0 delay.</b>
</div>

In [ ]:
df_train_c1_gt = df_train_c1.groupby('departure_hour').mean().sort_values('delay')[0:10]

In [ ]:
sns.barplot(df_train_c1_gt.index, df_train_c1_gt['delay'])

<div class=cbox>
    <b>From the graph above, there is 1 time that has the lowest delay that is 21 or 9PM.</b>
</div>

In [ ]:
df_train_c1_delay = df_train_c1[['delay']]
df_train_c1 = df_train_c1.drop(columns='delay')

<div class=cbox>
    <b>Divide into two types data, Numerical and Categorical Data</b>
</div>

In [ ]:
numerical_data = df_train_c1.select_dtypes(include = [np.number]).columns
categorical_data = df_train_c1.select_dtypes(exclude = [np.number]).columns

<div class=title1>
    <b><font size='2'>Create a model to estimate the delay duration ( Linear Regression )</font></b>
</div>

In [ ]:
datasets_dummies = pd.get_dummies(df_train_c1[categorical_data], drop_first=True)

In [ ]:
df_train_c1

In [ ]:
df_model = pd.concat([df_train_c1[numerical_data], datasets_dummies, df_train_c1_delay], axis = 1)
df_model.head()

In [ ]:
datasets_dummies = pd.get_dummies(df_test[categorical_data], drop_first=True)

In [ ]:
df_model_test = pd.concat([df_test[numerical_data], datasets_dummies, df_test['delay']], axis = 1)
df_model_test.head()

In [ ]:
x = df_model.iloc[:,0:len(df_model.columns)-1]
y = df_model.iloc[:,len(df_model.columns)-1].values.reshape(-1,1)
x = preprocessing.normalize(x)

<div class=cbox>
    <b>Check Data Correlation</b>
</div>

In [ ]:
np.abs(df_model.corr(method= 'pearson')['delay']).sort_values(ascending=False)[1:11]

In [ ]:
regression = LinearRegression(normalize=True)
regression.fit(x, y)

In [ ]:
x_test = df_model_test.iloc[:,0:len(df_model.columns)-1]
y_test = df_model_test.iloc[:,len(df_model.columns)-1].values.reshape(-1,1)
x_test = preprocessing.normalize(x_test)

In [ ]:
y_pred = regression.predict(x_test)

In [ ]:
print("MSE Y Test = ", mean_squared_error(y_pred,y_test))

In [ ]:
res = y_pred - y_test

plt.hist(res, bins=100)
plt.show()

In [ ]:
data_lin_res = df_test[['delay']].copy()
data_lin_res['delay_model'] = y_pred

In [ ]:
data_lin_res.head(20)

<div class=title1>
    <b><font size='2'>Create a model to predict delay > 60 min ( Logistic regression and another supervised model you
choosing )</font></b>
</div>

<div class=title1>
    <b><font size='2'>Logistic Regression</font></b>
</div>

In [ ]:
x = df_model.iloc[:,0:len(df_model.columns)-1]
y = df_model.iloc[:,len(df_model.columns)-1]

In [ ]:
log_regression = LogisticRegression(solver='lbfgs',multi_class='auto')
log_regression.fit(x, y)

In [ ]:
x_test = df_model_test.iloc[:,0:len(df_model.columns)-1]
y_test = df_model_test.iloc[:,len(df_model.columns)-1]

In [ ]:
y_pred = regression.predict(x_test)

In [ ]:
print("MSE Y Test = ", mean_squared_error(y_pred,y_test))

In [ ]:
data_log_res = df_test[['delay']].copy()
data_log_res['delay_model'] = y_pred

In [ ]:
data_log_res.head(20)

<div class=title1>
    <b><font size='2'>XGBoost</font></b>
</div>

In [ ]:
x = df_model.iloc[:,0:len(df_model.columns)-1]
y = df_model.iloc[:,len(df_model.columns)-1]

In [ ]:
xgb = XGBClassifier()
xgb.fit(x, y)

In [ ]:
x_test = df_model_test.iloc[:,0:len(df_model.columns)-1]
y_test = df_model_test.iloc[:,len(df_model.columns)-1]

In [ ]:
y_pred = regression.predict(x_test)

In [ ]:
print("MSE Y Test = ", mean_squared_error(y_pred,y_test))

In [ ]:
res = y_pred - y_test

plt.hist(res, bins=100)
plt.show()

In [ ]:
data_xgb_res = df_test[['delay']].copy()
data_xgb_res['delay_model'] = y_pred

In [ ]:
data_xgb_res.head(20)